
# Unity Catalog – Criação de SCHEMAS por camada (Bronze / Silver / Gold)

Este notebook cria **schemas nomeados por aplicação e camada** no **Unity Catalog**, seguindo o padrão:

- Prefixos de camada:
  - `b_*` → **Bronze**
  - `s_*` → **Silver**
  - `g_*` → **Gold**
- Convenção de nomes: `catalog.schema.tabela` (ex.: `prd.b_dm_callcenter.minha_tabela`)
- Cada **schema** recebe uma **MANAGED LOCATION** no ADLS organizada por camada:
  - `abfss://<container>@<storage>.dfs.core.windows.net/<camada>`

> ✅ **Pré-requisitos**
> - Workspace com **Unity Catalog** habilitado
> - Permissão para **CREATE SCHEMA** no catálogo
> - **RBAC** no **Storage Account** que hospeda o container
>
> 🔎 **O que este notebook faz**
> 1. Valida o catálogo informado.
> 2. Cria schemas em lote com **MANAGED LOCATION** por camada.
> 3. Faz *fallback* (criação sem location) caso falte permissão.
> 4. Mostra um **resumo** do que foi criado.


## Parâmetros editáveis
Ajuste o catálogo, storage, container e a lista de schemas que deseja criar.


In [ ]:
# ===================== PARÂMETROS EDITÁVEIS =====================
catalog_name         = "prd"                 # catálogo UC (ex.: prd)
storage_account_name = "stacjtecprd001"      # nome do Storage Account (ADLS Gen2)
container_name       = "ctcjtecprd001"       # nome do container
# Schemas a criar (pode adicionar/remover conforme necessidade)
schemas_desejados = [
    "b_dm_callcenter",
    "s_dm_callcenter",
    "g_dm_callcenter",
]

print("Catálogo:", catalog_name)
print("Storage :", storage_account_name)
print("Container:", container_name)
print("Schemas :", schemas_desejados)

## Funções auxiliares
- **`prefixo_para_camada`**: infere `bronze|silver|gold` a partir do prefixo do schema.
- **`managed_location`**: monta a URL ABFS para **MANAGED LOCATION** por camada.

In [ ]:

from pyspark.sql.utils import AnalysisException

def prefixo_para_camada(schema_name: str) -> str:
    """Retorna a camada (bronze/silver/gold) com base no prefixo do schema."""
    if schema_name.startswith("b_"):
        return "bronze"
    if schema_name.startswith("s_"):
        return "silver"
    if schema_name.startswith("g_"):
        return "gold"
    raise ValueError(f"Schema '{schema_name}' inválido: prefixo deve ser 'b_', 's_' ou 'g_'.")

def managed_location(layer: str) -> str:
    """Monta o caminho ABFS para a MANAGED LOCATION da camada."""
    return f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{layer}"



## ▶️ Execução
1. Valida o catálogo
2. Cria os schemas com **MANAGED LOCATION** (ou fallback)
3. Exibe um **resumo**


In [ ]:

print("==> Validando catálogo...")
try:
    spark.sql(f"USE CATALOG {catalog_name}")
    print(f"[OK] Catálogo encontrado: {catalog_name}")
except AnalysisException as e:
    raise Exception(
        f"Catálogo '{catalog_name}' não encontrado ou sem permissão. "
        f"Crie/ajuste antes de prosseguir."
    ) from e

print("==> Criando schemas com MANAGED LOCATION por camada...")
criadas, fallback = [], []
for schema in schemas_desejados:
    layer = prefixo_para_camada(schema)   # bronze/silver/gold
    mloc  = managed_location(layer)       # URL ABFS para MANAGED LOCATION
    fqn   = f"{catalog_name}.{schema}"    # nome completo do schema
    try:
        spark.sql(f"""
            CREATE SCHEMA IF NOT EXISTS {fqn}
            MANAGED LOCATION '{mloc}'
        """)
        criadas.append((fqn, layer, mloc, "managed"))
        print(f"[✓] {fqn:<45} | camada={layer:<6} | MANAGED LOCATION={mloc}")
    except AnalysisException as e:
        # Fallback: cria sem caminho (usa o storage root do catálogo)
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {fqn}")
        fallback.append((fqn, layer, getattr(e, "desc", str(e))))
        print(f"[!] {fqn:<45} | camada={layer:<6} | sem MANAGED LOCATION (fallback). Motivo: {getattr(e,'desc',e)}")

print("\n==> Resumo")
for fqn, layer, mloc, _ in criadas:
    print(f" - {fqn}  →  MANAGED LOCATION = {mloc}  (camada={layer})")
for fqn, layer, err in fallback:
    print(f" - {fqn}  →  criado SEM managed location (fallback). camada={layer}. "
          f"Ajuste permissões e rode: ALTER SCHEMA {fqn} SET MANAGED LOCATION '<path>';")



## 🔎 Validação (opcional)
Use os comandos abaixo para inspecionar as MANAGED LOCATIONs e confirmar as permissões.


In [ ]:
# Lista os schemas do catálogo
spark.sql(f"SHOW SCHEMAS IN {catalog_name}").display()

# Mostra detalhes dos schemas criados (inclui localização gerenciada quando definida)
for s in schemas_desejados:
    fqn = f"{catalog_name}.{s}"
    print(f"\n--- DESCRIBE SCHEMA EXTENDED {fqn} ---")
    spark.sql(f"DESCRIBE SCHEMA EXTENDED {fqn}").display()
